**30 November 2020**

# Word embeddings

## Admin

* Problem set 11 (the last one!)
    * To be released later this week
    * Due Tuesday, 12/8, at 11:59pm
* Read Nelson for Wednesday
    * Fourth and final reading response due by the evening of Tuesday, 12/15
* Need to discuss readings and/or lecture topics for next week
    * Post your ideas on Campuswire (threads already exist there) by Wednesday evening
* Exam and final project description and guidelines coming this week
    * Exam due date TBD this week
    * Exam released one week before due date

## Words as vectors

* We've previously represented **documents** as vectors
    * Usually, vectors of word counts
    * Or, with topic models, as vectors of topics
* Limits of words, often discussed
    * In brief: some words are more alike than others, but word counts don't reflect this fact
    * Topic models helped to address this problem
    * Today, a different approach: **word embeddings**
* We might like to have a vector for each **word**, which we could in turn use to build our document vectors
    * This will be an *unsupervised* task. We want to produce vectors that place similar words close together in vector space, but we don't want to use human-labled data to do so. We want instead to observe patterns of use in a large corpus.
* One approach (not pursued here): Truncated SVD on sliding windows over words
    * Group all the resulting bags of words by their 'center' word
    * Reduce resulting "doc"-term matrix for each word via truncated SVD
        * Like latent semantic analysis, but oriented around words rather than documents
    * Relatively fast to train and decent performance
    * Limited by linear relationships and lack of semantic connections
* Language models
    * Large neural language models achieve state-of-the-art performance on many (all?) NLP tasks
    * These models generally seek to predict a word (or other element of a text sequence) on the basis of the words around it
    * General architecture is:
        * Embedding layer. Counts of words transformed into contextual vectors. Initially, weights are random. In the case of word embeddings, we want to learn the proper weights as our output objective.
        * Intermediate (hidden) layer. Neural layers that can learn non-linear relationships between words and embeddings.
        * Output layer. Optimize an objective function (for example, highest probability of next word, given an input sequence).
    * `word2vec` (2013) showed that we can do without the intermediate layer if we just want to produce word embeddings
        * This simplifies the computation, so we can use *much* more input text
* CBOW vs. skip-gram
    * `word2vec` can use two different models: Continuous bag of words (CBOW) and skip-gram
    * CBOW is more "traditional." Objective is to predict a word given the *n* words before and after it.
    * Skip-gram "flips" the prediction task. Try to predict the *n* contextual words, given a seed word.
    * CBOW is faster to train and better captures *syntax* (plurals are closer to other plurals in vector space)
    * Skip-gram is slower to train and better captures *semantic* relationships ('king' moves  closer to 'prince' than to 'kings')
* Training is slow either way, so using pretrained embeddings is standard practice, but ...
    * Embedded relationships and biases of training set
    * Change over time
    * Polysemy remains within the embedding
        * "Class" vector has components of education and of social status, for example

## Let's *use* some embeddings

We're not going to train them, because that's slow and requires lots of corpus text. If we have time, we'll at least mention transfer learning in a future lecture. Transfer learning allows us to use a modest amount of new training data to update a larger, pretrained model.

### We need a `spaCy` medium or large model

The small models that we used in the previous lecture do not include vectors. Uncomment and run the cell below to download and install the large model, which includes `GloVe` embeddings (`GloVe` is an alternative to `word2vec`).

In [1]:
#!python -m spacy download en_core_web_lg

In [2]:
import spacy
nlp = spacy.load("en_core_web_lg") # Note '_lg' = large model

### Word similarities

In [3]:
# How similar are the words 'king' and 'queen' in Web text?
from sklearn.metrics.pairwise import cosine_similarity

print("king -> queen similarity:", cosine_similarity(
    nlp.vocab['king'].vector.reshape(1,-1), # reshape into a 2d vector with single row
    nlp.vocab['queen'].vector.reshape(1,-1)
).item())

king -> queen similarity: 0.7252609729766846


### Vector math

In [4]:
# Vector math: king - man + woman -> closer to queen?
king_to_queen_vector = nlp.vocab['king'].vector - nlp.vocab['man'].vector + nlp.vocab['woman'].vector
print("king - man + woman -> queen similarity:", cosine_similarity(
    king_to_queen_vector.reshape(1,-1), 
    nlp.vocab['queen'].vector.reshape(1,-1)
).item())

king - man + woman -> queen similarity: 0.7880845069885254


In [5]:
# Try the same with Paris and Berlin
print("Paris -> Berlin:                   ", cosine_similarity(
    nlp.vocab['Paris'].vector.reshape(1,-1), # reshape into a 2d vector with single row
    nlp.vocab['Berlin'].vector.reshape(1,-1)
).item())

paris_to_berlin_vector = nlp.vocab['Paris'].vector - nlp.vocab['France'].vector + nlp.vocab['Germany'].vector
print("Paris - France + Germany -> Berlin:", cosine_similarity(
    paris_to_berlin_vector.reshape(1,-1), 
    nlp.vocab['Berlin'].vector.reshape(1,-1)
).item())

Paris -> Berlin:                    0.5989030599594116
Paris - France + Germany -> Berlin: 0.7547403573989868


So, we can use vectors on these kinds of "analogy" tasks. How could we find the *n* most similar words to a given vector?

### Corpus-wide similarities

In [6]:
%%time
# How large is our model?
string_count = 0
vector_count = 0
for string in nlp.vocab.strings:
    string_count+=1
    if nlp.vocab[string].has_vector:
        vector_count+=1
print("Vocab size:", string_count)
print("Has vectors:", vector_count)

Vocab size: 1476295
Has vectors: 684754
CPU times: user 37.3 s, sys: 592 ms, total: 37.9 s
Wall time: 41.1 s


Create a matrix of all our embeddings. This matrix will have 684,754 rows (one for each word that has an embedding in our model) and 300 columns (the number of dimensions in the embedding representation).

We initialize the output matrix with zeros, since it's much faster to update an existing numpy matrix than it is to append to it.

Note that there's an easier way to do this in spaCy, but we're showing the details for clarity ...

In [7]:
%%time
# Make a word-vector martrix with labels
import numpy as np
vector_matrix = np.zeros([vector_count,nlp.vocab.vectors_length]) # Initialize the output matrix
counter = 0
vocab_dict = {} # Dictionary to hold word index positions in the matrix
vocab_list = [] # List to hold words in order
for string in nlp.vocab.strings: # iterate over the strings in our model
    if nlp.vocab[string].has_vector: # only want the ones with embeddings
        vocab_dict[string] = counter # record positiuon of this word
        vocab_list.append(string) # add to our list of words
        # normalize the vector and update matrix
        vector_matrix[counter] = nlp.vocab[string].vector/nlp.vocab[string].vector_norm
        counter+=1 # increment counter
print("Matrix shape:", vector_matrix.shape)

Matrix shape: (684754, 300)
CPU times: user 17.5 s, sys: 960 ms, total: 18.5 s
Wall time: 19.3 s


You might be tempted to iterate over the rows in the output matrix, calculating the similarity between each one and your target vector. This is slow. Instead, we'll use matrix math.

The **dot product** of two 1D vectors is the sum of their element-wise products. If the vectors are (L2) normalized, their dot product = their cosine similarity.

In [8]:
# Calculate similarities to 'king' via dot products
similarities = np.dot(vector_matrix, vector_matrix[vocab_dict['king']])
similarities.shape

(684754,)

In [9]:
# Get 10 most-similar words to 'king'
# argsort returns lowest to highest, so we get the last 10, then reverse
top_n = np.argsort(similarities)[-10:][::-1] 
for i in top_n:
    print(vocab_list[i], similarities[i])

king 1.000000026584885
KIng 1.000000026584885
KING 1.000000026584885
King 1.000000026584885
KINGS 0.787661407451134
Kings 0.787661407451134
kings 0.787661407451134
prince 0.7337737081547175
Prince 0.7337737081547175
PRINCE 0.7337737081547175


Note that the underlying vectors are obviously not case sensitive.

In [10]:
# Most similar to our king - man + woman vector
# Also, make this prettier and with less repetition

def print_most_similar(matrix, vector, vocab_list, n=20):
    sim = np.dot(matrix, vector/np.linalg.norm(vector))
    sorted_sims = np.argsort(sim)[-n:][::-1] #argsort return lowest to highest, so we get the last n, then reverse
    last_word = ''
    for i in sorted_sims:
        current_word = vocab_list[i].lower()
        if current_word != last_word:
            print(f"{current_word}     \t{sim[i]:.4}")
            last_word = current_word
            
print_most_similar(vector_matrix, king_to_queen_vector, vocab_list)

king     	0.8024
queen     	0.7881
prince     	0.6401
kings     	0.6209
princess     	0.6126
royal     	0.5801
throne     	0.5787


### Pure spacy

Do the same thing, but entirely within spacy ...

In [11]:
best = nlp.vocab.vectors.most_similar(king_to_queen_vector.reshape(1,-1), n=20)

In [12]:
# Examine the output
best

(array([[13176088972490086564, 14826469074451677028,  7464393751932445219,
          7102492827649024548, 10168488388102651113,  4176741725343376093,
          5247273317732208552, 12278543830867659210, 11742085837932180620,
          4527521648030784477,  5578855234836721309,  3708855936003160130,
          6599815902708227193,  8637828561746775750, 13482751041672084183,
         10227307316083268331,  9482215189757771475, 10275277408386923532,
           982181824006872346, 13488378799514498820]], dtype=uint64),
 array([[391588,   2183,   3150,  27270,   5310,   6026,  59856,  11900,
          94889,   7474,   6603,  58142,   9576,   9118,  91941,   8298,
          95982,   7927,   9741,  19962]], dtype=int32),
 array([[0.8024, 0.8024, 0.8024, 0.8024, 0.7881, 0.7881, 0.7881, 0.6401,
         0.6401, 0.6401, 0.6209, 0.6209, 0.6209, 0.6126, 0.6126, 0.6126,
         0.5801, 0.5801, 0.5801, 0.5787]], dtype=float32))

Note that the output format of spacy's `most_similar()` function is a tuple that contains three items: an array of word hashes (unique IDs), an array of index positions in the vector matrix, and an array of similarity values.

In [13]:
# Print top matches
hashes, lines, scores = best
last_word = ''
for i in range(len(hashes[0])):
    current_word = nlp.vocab[hashes[0][i].item()].text.lower()
    if current_word != last_word:
        print(f'{current_word}     \t{scores[0][i]:.4}')
        last_word = current_word

king     	0.8024
queen     	0.7881
prince     	0.6401
kings     	0.6209
princess     	0.6126
royal     	0.5801
throne     	0.5787


## Document similarities using word embeddings

In [14]:
# Doc similarities
from sklearn.feature_extraction.text import TfidfVectorizer

# Set up sample sentences
sentences = [
    'The young cat is smaller than the old dog.',
    'The baby kitten is tinier than the elderly hound.',
    'Not all sentences are about animals, nor about their properties.',
    'Some paragraphs concern a cat and its traits.'
]

# process sentences into spacy nlp objects
docs = []
for sentence in sentences:
    tokens = nlp(sentence)
    docs.append(tokens)
    
def remove_noninformative_tokens(doc):
    '''
    Takes a spacy-processed document.
    Returns a list of spacy token objects without stopwords, punctuation, or embedding-less tokens.
    '''
    culled = [token for token in doc if not (token.is_stop or token.is_punct) and token.has_vector]
    return(culled)

def embedding_similarity(X,Y):
    '''
    Takes two lists of spacy token objects.
    Returns cosine similarity between their embedding representations.
    '''
    mean_vector_X = np.mean([token.vector for token in X], axis=0)
    mean_vector_Y = np.mean([token.vector for token in Y], axis=0)
        # note below reshape 1D -> 2D array. 2D version is 1 row x n columns
    cos_similarity = cosine_similarity(mean_vector_X.reshape(1, -1), mean_vector_Y.reshape(1, -1))
    return(cos_similarity.item()) # use .item() to get single value from array
    
def token_similarity(X,Y):
    '''
    Takes two lists of spacy tokens.
    Returns cosine similarity between their tokens.
    '''
    doc1 = ' '.join([token.text for token in X])
    doc2 = ' '.join([token.text for token in Y])
    vectorizer = TfidfVectorizer(use_idf=False)
    vectors = vectorizer.fit_transform([doc1, doc2])
    return(cosine_similarity(vectors[0], vectors[1]).item())
    
for i, doc in enumerate(docs):
    doc1_nostops = remove_noninformative_tokens(doc)
    for j in range(len(docs)):
        if j>i:
            doc2_nostops = remove_noninformative_tokens(docs[j])
            print(f"Doc {i}: {docs[i]}")
            print(f"Doc {j}: {docs[j]}")
            print(f"   Built-in similarity: {doc.similarity(docs[j]):.3}")
            print(f"   Hand similarity:     {embedding_similarity(doc, docs[j]):.3}")
            print(f"   Embed sim, no stops: {embedding_similarity(doc1_nostops, doc2_nostops):.3}")
            print(f"   Token sim, no stops: {token_similarity(doc1_nostops, doc2_nostops):.3}")
            print()

Doc 0: The young cat is smaller than the old dog.
Doc 1: The baby kitten is tinier than the elderly hound.
   Built-in similarity: 0.923
   Hand similarity:     0.923
   Embed sim, no stops: 0.782
   Token sim, no stops: 0.0

Doc 0: The young cat is smaller than the old dog.
Doc 2: Not all sentences are about animals, nor about their properties.
   Built-in similarity: 0.789
   Hand similarity:     0.789
   Embed sim, no stops: 0.492
   Token sim, no stops: 0.0

Doc 0: The young cat is smaller than the old dog.
Doc 3: Some paragraphs concern a cat and its traits.
   Built-in similarity: 0.849
   Hand similarity:     0.849
   Embed sim, no stops: 0.639
   Token sim, no stops: 0.224

Doc 1: The baby kitten is tinier than the elderly hound.
Doc 2: Not all sentences are about animals, nor about their properties.
   Built-in similarity: 0.719
   Hand similarity:     0.719
   Embed sim, no stops: 0.364
   Token sim, no stops: 0.0

Doc 1: The baby kitten is tinier than the elderly hound.
Doc 